In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import imdb

# Load dataset
(train_data, train_labels), _ = imdb.load_data(num_words=10000)

# Vectorize sequences
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = vectorize_sequences(train_data)

# Define custom MAE metric
class MyMetrics(keras.metrics.Metric):
    def __init__(self, name="mae", **kwargs):
        super().__init__(name=name, **kwargs)
        self.mae_sum = self.add_weight(name="mae_sum", initializer="zeros")
        self.total_samples = self.add_weight(
            name="total_samples", initializer="zeros", dtype="int32")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)

        mae = tf.reduce_sum(tf.abs(y_true - y_pred))
        self.mae_sum.assign_add(mae)
        num_samples = tf.shape(y_pred)[0]
        self.total_samples.assign_add(num_samples)

    def result(self):
        return self.mae_sum / tf.cast(self.total_samples, tf.float32)

    def reset_state(self):
        self.mae_sum.assign(0.)
        self.total_samples.assign(0)

# Build the model
model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

# Compile the model with the custom MAE metric
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy", MyMetrics()])  # Add custom MAE metric here

# Train the model
history_original = model.fit(train_data, train_labels,
                             epochs=20, batch_size=512, validation_split=0.4)

# Accessing MAE values during training:
# After the model finishes training, you can access the history object to see the MAE values.
print(history_original.history)

# If 'mae' is present, it will be printed like this:
if 'mae' in history_original.history:
    print("MAE during training:")
    print(history_original.history['mae'])
else:
    print("MAE not found in the history.")


# After training, evaluate the model to get the MAE:
# Evaluate the model on the training data (or use a separate test dataset if available)
loss, accuracy, mae_value = model.evaluate(train_data, train_labels, verbose=0)

# Print the MAE value after training
print("MAE after training:", mae_value)